In [ ]:
!pip uninstall keras -y
!pip install keras==2.2.4

!pip uninstall tensorflow -y
!pip install tensorflow==1.14.0

!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
import numpy as np
import pandas as pd
import os

import nltk
from nltk.tokenize import WordPunctTokenizer
nltk.download('punkt')

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_1 = pd.read_csv("/kaggle/input/entity-annotated-corpus/ner_dataset.csv", encoding="latin1")
df_1 = df_1.fillna(method="ffill")

df_1['Sentence #'] = df_1['Sentence #'].apply(lambda x: x.split(': ')[1]).astype(int)
df_1.drop(['POS','Tag'],axis=1,inplace=True)

df_1.columns = ['Sentence','Word']

df_1.head()

In [ ]:
def annotater(arg):
    caps = first = small = misc = 0
    n = len(arg)
    
    for i in range(n):
        if 65<=ord(arg[i])<=90:
            if i==0:
                first = 1
            caps += 1
            
        elif 97<=ord(arg[i])<=122:
            small += 1
            
        else:
            misc += 1
            
    if small==n:
        return 's'
    elif caps==n:
        return 'c'
    elif first + small==n:
        return 'f'
    elif misc>0:
        return 'm'
    else:
        return 'f'
    
df_1['Target'] = df_1['Word'].apply(annotater)
df_1.head()

In [ ]:
def reader(filename):
    file =  open(filename)
    lines = file.readlines()
    text = ''
    for i in lines:
        text += i
    return text

english = reader("/kaggle/input/english/english.txt")
english_sentences = english.split('.')

In [ ]:
def tokenizer(arg):
    arg += '.'
    tokenizer = WordPunctTokenizer() 
    return tokenizer.tokenize(arg)

In [ ]:
sentences = []
numbers = []
for n,i in enumerate(english_sentences):
    tokens = tokenizer(i)
    for i in tokens:
        sentences.append(i)
    for i in range(len(tokens)):
        numbers.append(47960+n)

In [ ]:
df2 = pd.DataFrame(numbers,columns=['Sentence'])
df2['Word'] = sentences
df2['Target'] = df2['Word'].apply(annotater)

df = pd.concat([df_1,df2],axis=0)
df['Word'] = df['Word'].apply(lambda x: x.lower())
df.head()

In [ ]:
word2idx = {}

for n,i in enumerate(df['Word'].unique()):
    word2idx[i] = n+2

word2idx["PAD"]= 0
word2idx["UNK"]=1
   
idx2word = {v:k for k,v in word2idx.items()}

In [ ]:
text2tag = {"P":0,'f':1,'c':2,'s':3,'m':4}

tag2text = {0:"P",1:'f',2:'c',3:'s',4:'m'}

In [ ]:
def encode(arg):
    if arg=='f':
        return 1
    elif arg=='c':
        return 2
    elif arg=='s':
        return 3
    elif arg=='m':
        return 4

df['Target'] = df['Target'].apply(encode)

In [ ]:
def get_next(n):
    values = df[df['Sentence']==n][['Word','Target']].values
    return values[:,0],values[:,1]

X_word = []
y_word = []

for i in tqdm(range(df['Sentence'].values[-1])):
    if i+1==22480:
        continue
    x_,y_ = get_next(i+1)
    X_word.append(np.array([word2idx[i] for i in x_]))
    y_word.append(np.array(y_))

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [ ]:
MAX_LEN = 80
MAX_LEN_CHAR = 10
EMBEDDING = 300
n_words = len(word2idx)
BATCH_SIZE = 32
EPOCHS = 5

X_word_pad = pad_sequences(X_word,MAX_LEN,padding="post", value=0)
y_word_pad = pad_sequences(y_word,MAX_LEN,padding="post", value=0)
y_word_pad = [to_categorical(i, num_classes=4+1) for i in y_word_pad]

In [ ]:
chars = set([char for word in list(word2idx.keys()) for char in word])

n_chars = len(chars)

print(n_chars)

In [ ]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [ ]:
X_char = []
for sentence in tqdm(X_word):
    sent_seq = []
    for i in range(MAX_LEN):
        word_seq = []
        for j in range(MAX_LEN_CHAR):
            try:
                word_seq.append(char2idx.get(idx2word[sentence[i]][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

In [ ]:
X_train_char, X_test_char, y_train, y_test = train_test_split(X_char, y_word_pad, test_size=0.2,random_state=42)

X_train_word, X_test_word, _, _ = train_test_split(X_word_pad, y_word_pad, test_size=0.2,random_state=42)

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense,concatenate, SpatialDropout1D, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [ ]:
word_in = Input(shape=(MAX_LEN,))

emb_word = Embedding(input_dim=n_words, 
                  output_dim=EMBEDDING, 
                  input_length=MAX_LEN,
                  mask_zero=True)(word_in)

char_in = Input(shape=(MAX_LEN,MAX_LEN_CHAR))

emb_char = TimeDistributed(Embedding(input_dim=len(char2idx), 
                  output_dim=20, 
                  input_length=MAX_LEN_CHAR,
                  mask_zero=True))(char_in)

char_enc = TimeDistributed(LSTM(units=80, return_sequences=False,
                                recurrent_dropout=0.3))(emb_char)

x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.2)(x)

model = Bidirectional(LSTM(units=50, 
                           return_sequences=True, 
                           recurrent_dropout=0.2))(x)

model = TimeDistributed(Dense(50, activation="relu"))(model)
crf = CRF(5)
out = crf(model)

model = Model([word_in, char_in], out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])
model.summary()

In [ ]:
history = model.fit([X_train_word,
                    np.array(X_train_char)],
                    np.array(y_train),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_split=0.1)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['crf_viterbi_accuracy'])
plt.plot(history.history['val_crf_viterbi_accuracy'])

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'])
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
pred = model.predict([X_test_word,
                    np.array(X_test_char)])
pred = np.argmax(pred, axis=-1)
y_true = np.argmax(y_test, -1)

y_true = [[tag2text[j] for j in i] for i in y_true]
pred = [[tag2text[j] for j in i] for i in pred]

In [ ]:
!pip install sklearn-crfsuite

In [ ]:
from sklearn_crfsuite.metrics import flat_classification_report

print(flat_classification_report(y_pred=pred, y_true=y_true))

In [ ]:
def decoder(labels,tokens):
    decoded = []
    for i in range(len(tokens)):
        if tag2text[labels[0][i]]=='f':
            tmp = list(tokens[i])
            tmp[0] = tmp[0].upper()
            decoded.append(''.join(tmp))
        elif tag2text[labels[0][i]]=='s':
            decoded.append(tokens[i])
        elif tag2text[labels[0][i]]=='c':
            decoded.append(tokens[i].upper())
        else:
            decoded.append(tokens[i])
    
    text = []
    for token in decoded:
        
        if token=='(':
            text.append(token)
        
        elif token not in '!"#$%&\'*)+-./:,;<=>?@[\\]^_`{|}~':
            text.append(token + ' ')
        else:
            if text[-1][-1]==' ':
                tmp = text.pop()
                text.append(tmp[:-1])
            
            text.append(token+' ')
            
    
    return ''.join(text)

def encode(text):

    tokens = WordPunctTokenizer().tokenize(text)
    
    encoded = []

    for token in tokens:
        if token in word2idx.keys():
            encoded.append(word2idx[token])
        else:
            encoded.append(1)

    encoded = encoded + [0 for x in range(80-len(encoded))]
    
    sent_seq = []
    for i in range(MAX_LEN):
        word_seq = []
        for j in range(MAX_LEN_CHAR):
            try:
                word_seq.append(char2idx.get(idx2word[encoded[i]][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(np.array(word_seq))        
    
    return np.array(encoded),np.array(sent_seq),tokens

In [ ]:
#text = input("Enter the Text for case conversion: ").lower()

text = 'uSIng mATPLotlib and SeaBORN LIBraries is one OF THE best IDea to do Eda.'.lower()

encoded_word,encoded_char,tokens = encode(text)

labels = np.argmax(model.predict([encoded_word.reshape(1,80),encoded_char.reshape(1,80,10)]),axis=-1)
text = decoder(labels,tokens)
print()
print("The converted text is: ",text)

In [ ]:
#text = input("Enter the Text for case conversion: ").lower()

text = 'people in india started using english as thier primary language next to hindi.'

encoded_word,encoded_char,tokens = encode(text)

labels = np.argmax(model.predict([encoded_word.reshape(1,80),encoded_char.reshape(1,80,10)]),axis=-1)
text = decoder(labels,tokens)
print()
print("The converted text is: ",text)